In [1]:
%%configure -f
{
 "executorCores": 4, 
 "executorMemory": "47696M",
 "conf": {"spark.default.parallelism": 1000,
          "spark.sql.shuffle.partitions": 1000,
          "spark.task.cpus": 1
         }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1568626660333_0001,pyspark,idle,Link,Link,


In [2]:
model_path = "s3://onai-ml-dev-eu-west-1/company2vec/model/nmf"
word2id_path = "s3://onai-ml-dev-eu-west-1/company2vec/common"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1568626660333_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from gensim.utils import lemmatize, tokenize
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from langdetect import detect
import string
from itertools import islice
import numpy as np
import smart_open
from collections import Counter, defaultdict
import math
import json

from pyspark.ml.feature import (HashingTF,
                                IDF,
                                Tokenizer,
                                StopWordsRemover,
                                CountVectorizer,
                                StringIndexer,
                                OneHotEncoderEstimator,
                                VectorAssembler,
                                VectorSizeHint,
                                StandardScaler,
                                PCA
                               )
from pyspark.ml import Pipeline, Transformer

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

from pyspark.ml.linalg import Vectors, VectorUDT

import smart_open
import faiss

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import graphene

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import os
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import numpy as np
import ssl
from elasticsearch.connection import create_ssl_context
from elasticsearch_dsl import Q

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

public_labels = {
    2444485: [9429145, 93196, 9870522, 7914436],
    645782: [380011, 392154, 5523392],
    875173: [237655379, 931146, 418171],
    813607: [100231, 357076],
    380011: [357076, 324490, 93339],
    882889: [127202, 5478606, 1025390, 645782],
    46329052: [189096, 915379, 46895276, 877008, 325290136, 20524024, 271958947, 21852987, 26363560, 110104150],
    106335: [319676, 377732, 61206100],
    877769: [874042, 780678, 953488, 883809, 875295, 874186, 874119],
    34049884: [882155, 30428758, 315394, 23037669, 27561],
    254287477: [883300, 30614595],
    5600108: [285880, 5433540, 878697, 35650, 688262, 226852452],
    876031: [410182, 874470, 874191, 879732, 5395336],
    883752: [880697, 65340486, 26320074, 883327, 1034090, 257501324],
    5920885: [1494039, 268074105, 34534627, 20385800, 23000545, 124640],
    628413: [272054403, 91192, 309779, 140283, 138644],
    364040: [381388, 184945, 874170, 42751952, 874183, 314896, 5126590, 841504],
    257501324: [35000, 47320264, 253748612, 85076655, 32053, 12144785, 8186273, 9934160, 557267859],
    695204: [35303, 274561, 683719, 370857, 561001, 874022, 387473, 394038, 8274485, 12188205],
    30614595: [883300, 254287477, 9956099, 380011, 27868703, 2386697, 126857],
    28224119: [26824144, 35023689, 386639, 393661],
    32449506: [875260, 27169270, 5629762, 26014489, 286119],
    233324810: [874864, 159230, 27860587, 35806, 876981],
    879554: [5487000, 236715563, 412090459, 875192, 278679, 180871, 22516334, 30274893, 5478907],
    5580060: [118474533, 1779941, 265154, 10405454],
    23335317: [7885406, 277444, 278933, 8983678, 874143, 409119],
    381865672: [874842, 410366, 873649, 275789, 882473],
    937352: [876758, 879422, 128861678, 6461781, 1859063],
    874119: [881803, 875849, 231533, 877769, 780678, 953488, 875295, 874042, 775001, 874186],
    680934: [135398, 882299, 668578, 4481676, 32012],
    2248076: [141249, 4975204, 98876, 21828553],
    3606442: [882547, 4509042, 20703565, 7435035, 94799, 288033, 359868],
    877235: [295170, 175265, 874520, 410366, 873649, 874977, 167945, 8090046],
    84148802: [275789, 30339992, 5533238, 5718736],
    5523392: [645782, 11809880, 1353107, 962864],
    413744: [409932, 875491, 109303666, 91638],
    314896: [330589, 34768, 184945],
    5126590: [874855, 631781, 364040, 831357, 874170],
    377732: [319676, 106772, 106335, 704634, 320105, 874828, 873861, 1519242, 533853947],
    874652: [377732, 319676, 106772, 704634, 312375, 278933, 874828, 4863668]
}
HOST = "https://vpc-datalake-es-dev-pkpwahah2cv4owesrl4g5xzafe.eu-west-2.es.amazonaws.com:443"
INDEX = "company"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
word2id = {}
id2word = {}
with smart_open.open(f"{word2id_path}/bow/word2id.csv", "r") as f:
    for line in f:
        word,idd = line.strip().split(",")
        word2id[word] = int(idd)
        id2word[int(idd)] = word

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
with smart_open.open(f"{model_path}/H/mat.np", "rb") as f:
    H = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
with smart_open.open(f"{model_path}/X/X_nmf.np", "rb") as f:
    X_nmf = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
with smart_open.open(f"{model_path}/X/company_info.np", "rb") as f:
    description = np.load(f, allow_pickle=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
def nmf_reranker(results):
    ''' We assume results[0] is the query company
    '''
    
    query_id = results[0].id
    query_vector = X_nmf[company2id[query_id]]
    
    ret = []
    for result in results[0:]:
        result_id = result.id
        if result_id not in company2id:
            continue

        result_vector = X_nmf[company2id[result_id]]
        d = np.linalg.norm(query_vector-result_vector)
        
        ret.append((d, result))
    
    return [el[1] for el in sorted(ret, key = lambda el: el[0])]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
def find_closest_revenue(candidate, revenue, ebitda):
    if candidate.revenue == 0 or candidate.ebitda == 0:
        return candidate.score * 100

    if candidate.revenue < revenue:
        score_revenue = 1 - candidate.revenue/revenue
    else:
        score_revenue = 1 - revenue/candidate.revenue

    if candidate.ebitda < ebitda:
        score_ebitda = 1 - candidate.ebitda/ebitda
    else:
        score_ebitda = 1 - ebitda/candidate.ebitda

    return (score_ebitda + score_revenue) / 2


def revenue_reranker(results):
    query = results[0]
    return sorted(results[1:21],
                  key=lambda x: find_closest_revenue(x,
                                                     query.revenue,
                                                     query.ebitda
                                                     )
                  )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
company2id = {}
for i,desc in enumerate(description):
    company2id[str(desc[0])] = i

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
class Company(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()
    country = graphene.String()
    sector = graphene.String()
    revenue = graphene.Float()
    ebitda = graphene.Float()
    num_employees = graphene.Int()
    score = graphene.String()
    region = graphene.String()
    description = graphene.String()
    company_type = graphene.String()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
client = Elasticsearch(HOST+"/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def obtain_result(doc_id, 
                  reranker=lambda ret: ret, 
                  max_size=100, 
                  region=None, 
                  keywords=None,
                  company_type=None,
                  revenue_min=-1,
                  revenue_max=-1
                 ):
    query = client.get(index='company', id=doc_id)

    # for key, val in query['_source'].items():
    #     print("%s: %s" % (key, str(val)))

    query = client.get(index='company', id=doc_id)

    revenue = query['_source']['latest_revenue']

    if revenue_min == -1:
        revenue_min = 0.1*revenue

    if revenue_max == -1:
        revenue_max = revenue/0.1

    filter_queries = [Q('range', latest_revenue={'gt': revenue_min, 'lt': revenue_max}),
                      Q('range', latest_ebitda_margin={'gt': 0})]

    if region:
        filter_queries.append(Q('match', region=region))

    if keywords:
        filter_queries.append(Q('match', company_description=keywords))

    if company_type:
        filter_queries.append(Q('match', company_type=query['_source']['company_type']))

    q = Q('bool',
          filter=filter_queries,
          should=[Q('more_like_this',
                    like=[{"_id": doc_id, "_index": INDEX}],
                    fields=[
                        "industry_class_code_desc",
                        "company_description",
                    ],
                    min_doc_freq=1,
                    min_term_freq=0,
                    max_query_terms=15,
                    include=True, )]
          )

    s = (
        Search(index='company')
            .using(client)
            .query(q)
            .params(search_type="dfs_query_then_fetch")
    )

    max_score = -1
    results = []

    for hit in s[0:max_size]:
        id = hit.meta.id

        score = 100 * (hit.meta.score / max_score)
        score = str(score)[0:5] + "%"
        cname = hit.company_name
        sector = (
            hit.industry_class_code_desc
            if hasattr(hit, "industry_class_code_desc")
            else ""
        )
        country = hit.country if hasattr(hit, "country") else ""
        company_type = hit.company_type if hasattr(hit, "company_type") else ""

        revenue = hit.latest_revenue if hasattr(hit, "latest_revenue") else 0
        num_employees = (
            hit.number_of_employees if hasattr(hit, "number_of_employees") else 0
        )

        region = hit.region if hasattr(hit, "region") else "n/a"

        description = hit.company_description if hasattr(hit, "company_description") else ""

        ebitda = hit.latest_ebitda if hasattr(hit, "latest_ebitda") else 0

        # weight = _kernel(base_vec, [revenue, num_employees], sigma=base_vec / 2)
        # print("weight=>", weight, base_revenue, revenue)

        results.append(
            Company(
                id=id,
                name=cname,
                revenue=revenue,
                ebitda=ebitda,
                sector=sector,
                num_employees=num_employees,
                country=country,
                score=score,
                region=region,
                description=description,
                company_type=company_type,
            )
        )

    return reranker(results)


def find_peer_in_result(results, peer):
    for i, company in enumerate(results):
        if peer == int(company.id):
            return i+1

    return -1


def add_ranks_in_tops(ranks, counts):
    for rank in ranks:
        if 0 < rank <= 10:
            counts[0] += 1
        elif 10 < rank <= 20:
            counts[1] += 1
        elif 20 < rank <= 50:
            counts[2] += 1
        elif 50 < rank <= 100:
            counts[3] += 1
        elif 100 < rank:
            counts[4] += 1
        else:
            counts[5] += 1
    return counts

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### More like this

In [16]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(query_id)
    ranks = []
    
    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 5, 12, 41, 13]

[645782, -1, -1, -1]

[875173, -1, -1, -1]

[813607, 13, 7]

[380011, 21, 16, 25]

[882889, -1, -1, -1, -1]

[46329052, 15, 4, -1, -1, 14, 63, 9, -1, -1, -1]

[106335, 20, 7, 3]

[877769, 2, 17, 5, 8, 42, 51, 47]

[34049884, 3, 9, -1, 18, 7]

[254287477, -1, -1]

[5600108, 27, -1, 42, -1, 66, -1]

[876031, 16, 12, 5, 10, 6]

[883752, 2, -1, -1, -1, -1, -1]

[5920885, -1, -1, 14, 2, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, 34, 25, 28, 9, 30, 12, 41, -1]

[257501324, 4, -1, 39, 41, -1, -1, -1, -1, -1]

[695204, 3, 6, 2, 5, 4, 10, -1, -1, -1, -1]

[30614595, -1, -1, 55, -1, -1, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, 24, 47, 22, -1, 55]

[233324810, -1, -1, 8, 6, 44]

[879554, -1, -1, 57, 8, -1, -1, -1, -1, -1]

[5580060, -1, 3, -1, -1]

[23335317, 3, 5, -1, 28, 10, -1]

[381865672, 29, -1, 25, 17, 5]

[937352, -1, -1, -1, -1, 70]

[874119, 29, 12, -1, -1, -1, -1, -1, 58, -1, -1]

[680934, -1, 56, 72, -1, -1]

[2248076, 45, 24, -1, 17]

[3606442, 8

### More like this 500

In [33]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers_500 = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(query_id, max_size=500)
    ranks = []
    
    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 5, 12, 41, 13]

[645782, -1, 256, 172]

[875173, 260, -1, 202]

[813607, 13, 7]

[380011, 21, 16, 25]

[882889, -1, -1, -1, 115]

[46329052, 15, 4, -1, -1, 14, 63, 9, -1, -1, -1]

[106335, 20, 7, 3]

[877769, 2, 17, 5, 8, 42, 51, 47]

[34049884, 3, 9, 145, 18, 7]

[254287477, 105, 431]

[5600108, 27, -1, 42, 166, 66, 116]

[876031, 16, 12, 5, 10, 6]

[883752, 2, -1, -1, 152, 142, -1]

[5920885, -1, -1, 14, 2, -1, -1]

[628413, -1, -1, -1, 154, 171]

[364040, 34, 25, 28, 9, 30, 12, 41, -1]

[257501324, 4, -1, 39, 41, -1, -1, -1, -1, -1]

[695204, 3, 6, 2, 5, 4, 10, -1, 247, -1, -1]

[30614595, 200, -1, 55, -1, -1, 261, 194]

[28224119, -1, -1, 112, 130]

[32449506, 24, 47, 22, 218, 55]

[233324810, -1, -1, 8, 6, 44]

[879554, -1, -1, 57, 8, -1, -1, -1, -1, -1]

[5580060, -1, 3, -1, -1]

[23335317, 3, 5, -1, 28, 10, -1]

[381865672, 29, 237, 25, 17, 5]

[937352, -1, 234, 107, -1, 70]

[874119, 29, 12, 135, 139, 155, 151, 118, 58, -1, -1]

[680934, -1, 56, 72, 335, -1]

[2248076

### More like this, revenue re-ranking

In [44]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(query_id, revenue_reranker)
    ranks = []
    
    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)

    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 1, 7, -1, 9]

[645782, -1, -1, -1]

[875173, -1, -1, -1]

[813607, 11, 2]

[380011, 2, 4, -1]

[882889, -1, -1, -1, -1]

[46329052, 8, 15, -1, -1, 2, -1, 10, -1, -1, -1]

[106335, 11, 10, 2]

[877769, 6, 3, 1, 14, -1, -1, -1]

[34049884, 3, 12, -1, 13, 7]

[254287477, -1, -1]

[5600108, -1, -1, -1, -1, -1, -1]

[876031, 3, 7, 2, 12, 14]

[883752, 16, -1, -1, -1, -1, -1]

[5920885, -1, -1, 18, 14, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, -1, -1, -1, 3, -1, 12, -1, -1]

[257501324, 11, -1, -1, -1, -1, -1, -1, -1, -1]

[695204, 1, 12, 14, 20, 17, 9, -1, -1, -1, -1]

[30614595, -1, -1, -1, -1, -1, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, -1, -1, -1, -1, -1]

[233324810, -1, -1, 19, 9, -1]

[879554, -1, -1, -1, 10, -1, -1, -1, -1, -1]

[5580060, -1, 18, -1, -1]

[23335317, 4, 16, -1, -1, 20, -1]

[381865672, -1, -1, -1, 5, 8]

[937352, -1, -1, -1, -1, -1]

[874119, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1]

[680934, -1, -1, -1, -1, -1]

[2248076, -1, -1, -1, 16]

[36064

### More like this, revenue re-ranking, 500

In [45]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers_500 = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(query_id, revenue_reranker, max_size=500)
    ranks = []
    
    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 1, 7, -1, 9]

[645782, -1, -1, -1]

[875173, -1, -1, -1]

[813607, 11, 2]

[380011, 2, 4, -1]

[882889, -1, -1, -1, -1]

[46329052, 8, 15, -1, -1, 2, -1, 10, -1, -1, -1]

[106335, 11, 10, 2]

[877769, 6, 3, 1, 14, -1, -1, -1]

[34049884, 3, 12, -1, 13, 7]

[254287477, -1, -1]

[5600108, -1, -1, -1, -1, -1, -1]

[876031, 3, 7, 2, 12, 14]

[883752, 16, -1, -1, -1, -1, -1]

[5920885, -1, -1, 18, 14, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, -1, -1, -1, 3, -1, 12, -1, -1]

[257501324, 11, -1, -1, -1, -1, -1, -1, -1, -1]

[695204, 1, 12, 14, 20, 17, 9, -1, -1, -1, -1]

[30614595, -1, -1, -1, -1, -1, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, -1, -1, -1, -1, -1]

[233324810, -1, -1, 19, 9, -1]

[879554, -1, -1, -1, 10, -1, -1, -1, -1, -1]

[5580060, -1, 18, -1, -1]

[23335317, 4, 16, -1, -1, 20, -1]

[381865672, -1, -1, -1, 5, 8]

[937352, -1, -1, -1, -1, -1]

[874119, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1]

[680934, -1, -1, -1, -1, -1]

[2248076, -1, -1, -1, 16]

[36064

### NMF reranker

In [34]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(query_id, nmf_reranker)
    ranks = []
    
    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 7, 99, 84, 20]

[645782, -1, -1, -1]

[875173, -1, -1, -1]

[813607, 98, 2]

[380011, 9, 8, 20]

[882889, -1, -1, -1, -1]

[46329052, 88, 51, -1, -1, 41, 35, 86, -1, -1, -1]

[106335, 34, 42, 2]

[877769, 18, 7, 12, 57, 13, 23, 74]

[34049884, 5, 51, -1, 82, 56]

[254287477, -1, -1]

[5600108, 97, -1, 50, -1, 85, -1]

[876031, 28, 17, 29, 2, 8]

[883752, 34, -1, -1, -1, -1, -1]

[5920885, -1, -1, 25, 8, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, 8, 16, 25, 21, 3, 26, 29, -1]

[257501324, 80, -1, 92, 78, -1, -1, -1, -1, -1]

[695204, 2, 25, 57, 97, 95, 18, -1, -1, -1, -1]

[30614595, -1, -1, 91, -1, -1, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, 35, 6, 32, -1, 93]

[233324810, -1, -1, 71, 88, 63]

[879554, -1, -1, 69, 79, -1, -1, -1, -1, -1]

[5580060, -1, 39, -1, -1]

[23335317, 24, 98, -1, 17, 93, -1]

[381865672, 91, -1, 96, 92, 24]

[937352, -1, -1, -1, -1, 68]

[874119, 91, 2, -1, -1, -1, -1, -1, 88, -1, -1]

[680934, -1, 6, 3, -1, -1]

[2248076, 3, 18, -1, 22]


### NMF reranker 500

In [35]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers_500 = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(query_id, nmf_reranker)
    ranks = []
    
    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)
    
    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 7, 99, 84, 20]

[645782, -1, -1, -1]

[875173, -1, -1, -1]

[813607, 98, 2]

[380011, 9, 8, 20]

[882889, -1, -1, -1, -1]

[46329052, 88, 51, -1, -1, 41, 35, 86, -1, -1, -1]

[106335, 34, 42, 2]

[877769, 18, 7, 12, 57, 13, 23, 74]

[34049884, 5, 51, -1, 82, 56]

[254287477, -1, -1]

[5600108, 97, -1, 50, -1, 85, -1]

[876031, 28, 17, 29, 2, 8]

[883752, 34, -1, -1, -1, -1, -1]

[5920885, -1, -1, 25, 8, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, 8, 16, 25, 21, 3, 26, 29, -1]

[257501324, 80, -1, 92, 78, -1, -1, -1, -1, -1]

[695204, 2, 25, 57, 97, 95, 18, -1, -1, -1, -1]

[30614595, -1, -1, 91, -1, -1, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, 35, 6, 32, -1, 93]

[233324810, -1, -1, 71, 88, 63]

[879554, -1, -1, 69, 79, -1, -1, -1, -1, -1]

[5580060, -1, 39, -1, -1]

[23335317, 24, 98, -1, 17, 93, -1]

[381865672, 91, -1, 96, 92, 24]

[937352, -1, -1, -1, -1, 68]

[874119, 91, 2, -1, -1, -1, -1, -1, 88, -1, -1]

[680934, -1, 6, 3, -1, -1]

[2248076, 3, 18, -1, 22]


In [26]:
with smart_open.open(f"{model_path}/eval/peers.tsv", "w") as f:
    header = "\t".join("peer_" + str(i + 1) for i in range(20))
    f.write(f"target_company\t{header}\n")
    for label in public_labels:
        s = ""
        query_id = label
        results = obtain_result(query_id, nmf_reranker, 500)
        for peer in results[:20]:
            t = json.dumps({'capiq_id': peer.id, 'name': peer.name, 'description': peer.description,
                           'revenue': peer.revenue, 'sector': peer.sector})
            s += t + "\t"
        f.write(s + "\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
for i, el in enumerate(zip(obtain_result(106335, nmf_reranker), obtain_result(106335))):
    print(f"    Rank {i+1}     ")
    print("------ NMF -----")
    print(el[0].id)
    print(el[0].name)
    print(el[0].description)
    print(el[0].revenue)
    print(el[0].num_employees)
    print(el[0].company_type)
    print(el[0].sector)
    
#     print("------ ES_more_like_this -----")
#     print(el[1].id)
#     print(el[1].name)
#     print(el[1].description)
#     print(el[1].revenue)
#     print(el[1].num_employees)
#     print(el[1].company_type)
    print("#"*80)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Rank 1     
------ NMF -----
106335
Ford Motor Company
Ford Motor Company designs, manufactures, markets, and services a range of Ford cars, trucks, sport utility vehicles, and electrified vehicles worldwide. It operates through three segments: Automotive, Mobility, and Ford Credit. The Automotive segment sells Ford and Lincoln vehicles, service parts, and accessories through distributors and dealers, as well as through dealerships to commercial fleet customers, daily rental car companies, and governments. The Mobility segment designs and builds mobility services; and provides self-driving systems development and vehicle integration, autonomous vehicle research and engineering, and autonomous vehicle transportation-as-a-service network development services. The Ford Credit segment primarily engages in vehicle-related financing and leasing activities to and through automotive dealers. It provides retail installment sale contracts for new and used vehicles; and direct financing lease

In [19]:
for i, el in enumerate(zip(obtain_result(46329052, nmf_reranker), obtain_result(46329052))):
    print(f"    Rank {i+1}     ")
    print("------ NMF -----")
    print(el[0].id)
    print(el[0].name)
    print(el[0].description)
    print(el[0].revenue)
    print(el[0].num_employees)
    print(el[0].company_type)
    print(el[0].sector)
    
#     print("------ ES_more_like_this -----")
#     print(el[1].id)
#     print(el[1].name)
#     print(el[1].description)
#     print(el[1].revenue)
#     print(el[1].num_employees)
    print("#"*80)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Rank 1     
------ NMF -----
46329052
Metro AG
Metro AG engages in the wholesale and food retail business. It operates 771 wholesale stores under the METRO and MAKRO brands primarily serving the needs of hotels, restaurants, catering companies, and independent retailers, as well as service providers and authorities. The company also develops digital solutions. In addition, it provides services in the areas of real estate, logistics, information technology, and advertising. The company was formerly known as METRO Wholesale & Food Specialist AG and changed its name to METRO AG in August 2017. Metro AG is based in Düsseldorf, Germany.
32707.5011096316
132293
Public Company
Groceries and related products - wholesale
################################################################################
    Rank 2     
------ NMF -----
7915896
Foncière Euris SA
Foncière Euris SA distributes food and specialized products in France and internationally. The company also distributes sporting goods

In [20]:
for i, el in enumerate(zip(obtain_result(5600108, nmf_reranker), obtain_result(5600108))):
    print(f"    Rank {i+1}     ")
    print("------ NMF -----")
    print(el[0].id)
    print(el[0].name)
    print(el[0].description)
    print(el[0].revenue)
    print(el[0].num_employees)
    print(el[0].company_type)
    print(el[0].sector)
    
#     print("------ ES_more_like_this -----")
#     print(el[1].id)
#     print(el[1].name)
#     print(el[1].description)
#     print(el[1].revenue)
#     print(el[1].num_employees)
    print("#"*80)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Rank 1     
------ NMF -----
5600108
Gestamp Automoción, S.A.
Gestamp Automoción, S.A. designs, develops, manufactures, and sells metal automotive components in Europe, North America, South America, and Asia. The company offers Body-in-White products, including large components and assembly parts, such as bonnets, roofs, doors, and mudguards, as well as other surface and assembly parts; and structural and crash-related elements that include floors, pillars, rails, and wheel arches. It also provides chassis comprises the under body of vehicles and systems, frames, and related parts, such as front and rear axles and couplings, control arms, and integrated couplings. In addition, the company offers mechanism parts, such as door hinges and door checks, powered systems, and driver controls. Gestamp Automoción, S.A. offers its products to manufacturers of light vehicles. The company was founded in 1997 and is headquartered in Abadiño, Spain. Gestamp Automoción, S.A. is a subsidiary of Ge